# **TuriCreate Image Classifier**
The following notebook includes an example of data import, model training and model saving for an image classifier.

In [ ]:
import turicreate as tc
import urllib
import tarfile
import coremltools
import save_models as sm
from skafossdk import *

In [ ]:
ska  = Skafos()

## **The Data**
The data for this example are 25,000 images, half cats and half dogs. We have loaded the images into an S3 bucket for ease. T

In [ ]:
# The code below specifies the S3 bucket to pull from
# downloads the compressed directory and then extracts the images
data_url = "https://s3.amazonaws.com/skafos.example.data/PetImages.tar.gz"
data_path = "PetImages.tar.gz"

# pull the tar
ska.log("Retrieving the images from online", labels = ['image_classifier'])
retrieve = urllib.request.urlretrieve(data_url, data_path)

# extract the file
ska.log("Images downloaded, extracting the images", labels = ['image_classifier'])
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

## **Data Load**
We use Turi Create's ```load_images``` function to recursively load all the images in the Dog and Cat sub-directories.
We then create the label column based on the directory it was.
Finally, we create a training data set to train on and a test data set to confirm model accuracy.



In [ ]:
# Load images (Note: you can ignore 'Not a JPEG file' errors)
ska.log("Loading the images locally", labels = ['image_classifier'])
data = tc.image_analysis.load_images('PetImages', with_path=True, ignore_failure = True)

# From the path-name, create a label column
data['label'] = data['path'].apply(lambda path: 'dog' if '/Dog' in path else 'cat')

# Make a train-test split
train_data, test_data = data.random_split(0.8)

## **Model Build and Save**
- Build the model by passing the training data, specifying the `target` column.
- Save the model by:
    - exporting to coreml
    - converting the weights to half-precision to decrease model size
    - compress the model
    - save the model to *Skafos*

Note: to see how we are saving these models, check out the `save_models.py` module. To read more about converting model weights to half-precision, click [here](https://developer.apple.com/documentation/coreml/reducing_the_size_of_your_core_ml_app).


In [ ]:
# Create the model
ska.log("Building the model", labels = ['image_classifier'])
model = tc.image_classifier.create(train_data, target='label')

In [ ]:
# Save the model for later use in Turi Create
coreml_model_name = 'image_classification.mlmodel'
res = model.export_coreml(coreml_model_name)

In [ ]:
# use coremltools utility function to convert model weights to half-precision
model_spec = coremltools.utils.load_spec(coreml_model_name)
model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

# save to Skafos
sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
								compressed_model = compressed_model,
								permissions = 'public')